In [191]:
from itertools import islice

import pandas as pd
import numpy as np

from openpyxl import load_workbook

DATA_FILE = "задание_dsc339a3a1-1431-4382-b898-9b0a9eef77e2.xlsx"

In [51]:
def load_data(sheetnum,
              fname=DATA_FILE, 
              col_names=("name", "gender", "document", "country"),
             ):
    wb = load_workbook(filename=DATA_FILE)
    ws = wb.worksheets[sheetnum]

    data = pd.DataFrame(islice(ws.values,1, None))
    data.columns = col_names
    return data

def split_name(df):
    data = df.copy()
    names = data["name"].str.split("\s+", expand=True)
    col_names = [f"name{i}" for i in names.columns]
    names.columns = col_names
    data.loc[:, col_names] = names[col_names]
    return data

 Загрузим данные и разделим имена на части.

In [59]:
fact = load_data(1)
test = load_data(2)
fact_data = split_name(fact)
test_data = split_name(test)

## Присвоение признака пола

Сколько уникальных фамилий в "фактическом" датасете:

In [57]:
fact_data["name0"].unique().shape[0]

5895

Сколько фамилий в "фактическом" датасете принадлежат людям обоих полов:

In [112]:
a = fact_data[["name0", "gender"]].drop_duplicates()
a = a.groupby("name0").count().reset_index()
twin_gender = a[a["gender"]>1]
len(twin_gender.index)


218

Присвоить признак пола по имени:

In [187]:
def match_gender(fact_data, test_data):
    columns = [c for c in fact_data.columns if c[:len("name")] == "name" and len(c)>len("name")]
    keys = ["name", "gender"] + columns
    matched = pd.DataFrame(dict(zip(keys, [[] for i in range(len(keys))])))
    for i,c in enumerate(columns):
        a = fact_data[[c, "gender"]].drop_duplicates().dropna()
        b = a.groupby(c).count().reset_index()
        twin_list = b[b["gender"]>1][c].tolist()
        lookup = a[~a[c].isin(twin_list)]
        idx = test_data[c].isin(a[c]) & \
                          (~test_data[c].isin(twin_list))
        if len(matched.index) > 0:
            for j in range(i):
                idx = idx & (~test_data[f"name{j}"].isin(matched[f"name{j}"]))
        combo = test_data[idx][["name"] + columns].drop_duplicates()
        res = pd.merge(combo, lookup, on=c)
        matched = pd.concat([matched, res])
    return matched

matched_gender = match_gender(fact_data, test_data)

У части тестовых записей отсутствуют аналоги в фактических данных

In [188]:
a = test_data[["name", "name0", "name1", "name2", "name3", "name4", "name5"]].drop_duplicates()
missing = a[~a["name"].isin(matched_gender["name"])]
missing

,name,name0,name1,name2,name3,name4,name5
12,АБДАЗИЗОВА ОМУРКАН,АБДАЗИЗОВА,ОМУРКАН,None,None,None,None
17,АБДИЛАМИТОВ МУМИН,АБДИЛАМИТОВ,МУМИН,None,None,None,None
20,АБДИНАЗИМ КЫЗЫ МУНАИМ,АБДИНАЗИМ,КЫЗЫ,МУНАИМ,None,None,None
26,АБДУГАНИЕВА МАЛОХАТ МУФТОХИДДИН КИЗИ,АБДУГАНИЕВА,МАЛОХАТ,МУФТОХИДДИН,КИЗИ,None,None
43,АБДУМАНАБОВ МУРАДЖОН СОВОНКУЛОВИЧ,АБДУМАНАБОВ,МУРАДЖОН,СОВОНКУЛОВИЧ,None,None,None
...,...,...,...,...,...,...,...
2431,ЩАННИКОВА НАТАЛЬЯ ЮРЬЕВНА,ЩАННИКОВА,НАТАЛЬЯ,ЮРЬЕВНА,None,None,None
2460,ЭРНАЗАРОВ САФАРАЛИ ГАЙИПОВИЧ,ЭРНАЗАРОВ,САФАРАЛИ,ГАЙИПОВИЧ,None,None,None
2461,ЭРНАФАСОВ ЭШМУРОД КУРОКБОЙ УГЛИ,ЭРНАФАСОВ,ЭШМУРОД,КУРОКБОЙ,УГЛИ,None,None
2464,ЭСЕНБАЕВА ГУЛСИНАЙ,ЭСЕНБАЕВА,ГУЛСИНАЙ,None,None,None,None


Попробуем найти близкие аналоги по косинусному расстоянию:

In [310]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def match_cosine(fact_data, test_data):
    matching_vec = fact_data["name"].unique()
    tfidf_vectorizer = TfidfVectorizer()
    sparse_matrix = tfidf_vectorizer.fit_transform(matching_vec)
    term_matrix = sparse_matrix.toarray()
    
    def _(to_match):
        match_trf = tfidf_vectorizer.transform([to_match]).toarray()
        match_cosine = cosine_similarity(term_matrix, match_trf)
        return (to_match, matching_vec[np.argmax(match_cosine), match_cosine])
    
    return [_(name) for name in test_data["name"]]


In [ ]:
a = list(zip(*match_cosine(fact_data, missing)))
cos_matched = pd.DataFrame({"test_name" : a[0], "fact_name" : a[1]})
b = fact_data.loc[fact_data["name"].isin(a[1]), ["name", "gender"]].drop_duplicates()
b = b.rename(columns={"name" : "fact_name"})
res = pd.merge(cos_matched, b, on="fact_name").drop(columns="fact_name")
res = res.rename(columns={"test_name" : "name"})

In [222]:
full_match = pd.concat([matched_gender[["name", "gender"]], res])

Проверим результат на базовую вменяемость. Поскольку таких случаев не слишком много, мы можем проверить их "на глаз" и если исключений не обнаружим, то заменим пол на противоположный для всех выявленных записей.

In [223]:
idx = (full_match["name"].str.contains(r"ОВНА\b") & (full_match["gender"] == "М")) | \
    (full_match["name"].str.contains(r"ОВИЧ\b") & (full_match["gender"] == "Ж"))
full_match[idx]

,name,gender
44,АДАМЯН РИТА ЗАВЕНОВНА,М
101,АМОЯН АРАМ САРГИСОВИЧ,Ж
124,АТАБЕКЯН ДЖИВАН НВЕРОВИЧ,Ж
174,БАКОЯН АНТАРАМ ГАГИКОВНА,М
193,БИЛЫК РОМАН ОДИЛОВИЧ,Ж
203,БОНДАРЬ СВЕТЛАНА ПОЛИКАРПОВНА,М
233,ГАРИБЯН АРЦВИК РАФИКОВНА,М
259,ГЕВОНДЯН АСТГИК АМАЯКОВНА,М
325,ДЬЯЧЕНКО ВАЛЕРИЯ АЛЕКСАНДРОВНА,М
340,ЖЕНИШБЕК КЫЗЫ МИЛАННА АЛЕКСАНДРОВНА,М


"ИВАНОВИЧ САМИРА ВИКТОРОВНА" явно определена правильно как женщина, что наводит нас на мысль о целесообразности дополнительного теста: 

In [226]:
idx2 = (full_match["name"].str.contains(r"ОВНА\b") & full_match["name"].str.contains(r"ОВИЧ\b"))
full_match[idx2]

,name,gender
896,ИВАНОВИЧ САМИРА ВИКТОРОВНА,Ж


Похоже, что Самира Викторовна уникальна, поэтому ее мы исключаем, а для остальных заменяем пол на противоположный:

In [224]:
exceptions = full_match[idx].copy()
exceptions = exceptions.drop(exceptions[exceptions["name"]=="ИВАНОВИЧ САМИРА ВИКТОРОВНА"].index)
gen_flip = {"Ж" : "М", "М" : "Ж"}
flipped_gen = [gen_flip[gen] for gen in exceptions["gender"]]
exceptions.loc[:, "gender"] = flipped_gen

full_match.loc[full_match["name"].isin(exceptions["name"]), "gender"] = exceptions["gender"]

In [225]:
full_match

,name,gender
0,АБАСОВ РАШАД РАДЖАБ ОГЛЫ,М
1,АБАСОВА УЛЬЯНА ИГОРЕВНА,Ж
2,АББАСОВ ДЖЕЙХУН ДАВУД ОГЛЫ,М
3,АББАСОВ ДУНЯМИН МАХМУД ОГЛЫ,М
4,АББАСОВ ЕЛЬНУР АРИЗ ОГЛЫ,М
...,...,...
242,ШАРИПОВ АБДИМАЛИК АЛИЖАНОВИЧ,М
243,ШИРАЛИЕВА СААДЕТ КАМИЛ КЫЗЫ,Ж
244,ЩАННИКОВА НАТАЛЬЯ ЮРЬЕВНА,Ж
245,ЭРНАЗАРОВ САФАРАЛИ ГАЙИПОВИЧ,М


## Присвоение признака страны

Наличие уникальных схем номеров документа для стран в фактических данных позволяет нам присвоить страну некоторой части наблюдений в тестовых данных. Эти уникальные схемы: 

In [305]:
# Find document patterns that are unique for a country
a = fact_data[["document", "country"]].drop_duplicates()
b = a.groupby(["document"]).count().reset_index()
unq_pat = b[b["country"]==1]
unq_doc_lookup = fact_data.loc[fact_data["document"].isin(unq_pat["document"]), ["document", "country"]].drop_duplicates()
unq_doc_lookup

,document,country
0,AP!!!!!!,АРМЕНИЯ
1,AB!!!!!!!,УЗБЕКИСТАН
2,СР!!!!!!!,УЗБЕКИСТАН
11,AZII!!!!!!,АЗЕРБАЙДЖАН
14,C!!!!!!!!,АЗЕРБАЙДЖАН
...,...,...
9281,PKGZAC!!!!!!,КИРГИЗИЯ
9341,CH!!!!!!!,УЗБЕКИСТАН
9397,!!!!!!!СО,АЗЕРБАЙДЖАН
9471,PUZBCB!!!!!!!,УЗБЕКИСТАН


Присвоим национальность части тестовых наблюдений:

In [306]:
a = test_data.copy()
a = a.drop(columns="country")
doc_matched = pd.merge(a, unq_doc_lookup)
doc_matched

,name,gender,document,name0,name1,name2,name3,name4,name5,country
0,АББАСОВ ЭЛЬФАГ ИБРАГИМ ОГЛЫ,None,P!!!!!!!!,АББАСОВ,ЭЛЬФАГ,ИБРАГИМ,ОГЛЫ,None,None,РОССИЯ
1,АБГАРЯН ДИАНА АРКАДЬЕВНА,None,AR!!!!!!,АБГАРЯН,ДИАНА,АРКАДЬЕВНА,None,None,None,АРМЕНИЯ
2,АЙРОЯН ГЕВОРГ ЭДУАРД,None,AR!!!!!!,АЙРОЯН,ГЕВОРГ,ЭДУАРД,None,None,None,АРМЕНИЯ
3,АМРОЯН КАДЖИК ГЕВОРГОВИЧ,None,AR!!!!!!,АМРОЯН,КАДЖИК,ГЕВОРГОВИЧ,None,None,None,АРМЕНИЯ
4,АНИСЯН АНАИТ КОРЮНОВНА,None,AR!!!!!!,АНИСЯН,АНАИТ,КОРЮНОВНА,None,None,None,АРМЕНИЯ
...,...,...,...,...,...,...,...,...,...,...
563,ЧЕРШУКОВА АНЖЕЛИКА ВЛАДИМИРОВНА,None,АВ!!!!!!!!,ЧЕРШУКОВА,АНЖЕЛИКА,ВЛАДИМИРОВНА,None,None,None,УЗБЕКИСТАН
564,ШИТОВА ОЛЬГА ВАСИЛЬЕВНА,None,ET!!!!!!,ШИТОВА,ОЛЬГА,ВАСИЛЬЕВНА,None,None,None,УКРАИНА
565,ЭВАЙЗОВ АДИЛ ТАНРЫВЕРДИ ОГЛЫ,None,CO!!!!!!!,ЭВАЙЗОВ,АДИЛ,ТАНРЫВЕРДИ,ОГЛЫ,None,None,АЗЕРБАЙДЖАН
566,ЭЙВАЗОВА СААДАТ ЭЛЬЧИН КЫЗЫ,None,VIIМЮ!!!!!!,ЭЙВАЗОВА,СААДАТ,ЭЛЬЧИН,КЫЗЫ,None,None,РОССИЯ


In [308]:
len(test_data.index) - len(doc_matched.index)

1932

In [341]:
# find the closest match for test name in each country group
from difflib import SequenceMatcher
from tqdm import tqdm


def similarity(word1, word2):
    return SequenceMatcher(None, word1, word2).ratio()

def country_matcher(to_match, method="max"):
    assert method in ("max", "mean"), f"Unknown method: {method}"
    meth = {"max" : np.max, "mean" : np.mean}[method]
    def _(country_grp):
        res = {"name0" : [], "similarity" : []}
        for name in tqdm(to_match):
            sims = [similarity(name, name2) for name2 in country_grp["name0"] if name != name2]
            res["name0"].append(name)
            res["similarity"].append(meth(sims))
        df = pd.DataFrame(res)
        return df
    return _
    
a = fact_data[["name0", "country"]].copy().dropna()
b = test_data.loc[~test_data["name"].isin(doc_matched["name"]), ["name0", "country"]].copy()

sims = a[["name0", "country"]].groupby("country").apply(country_matcher(b["name0"]))

  0%|                                          | 5/1932 [00:00<01:40, 19.18it/s]


KeyboardInterrupt: 

In [338]:
sims.reset_index()

,country,level_1,name0,similarity
0,АЗЕРБАЙДЖАН,0,АБАСОВ,0.923077
1,АЗЕРБАЙДЖАН,1,АБАСОВА,0.933333
2,АЗЕРБАЙДЖАН,2,АББАСОВ,0.933333
3,АЗЕРБАЙДЖАН,3,АББАСОВ,0.933333
4,АЗЕРБАЙДЖАН,4,АББАСОВ,0.933333
...,...,...,...,...
11587,УКРАИНА,1927,ЯКУБОВА,0.750000
11588,УКРАИНА,1928,ЯКУШЕВА,0.714286
11589,УКРАИНА,1929,ЯСИК,0.600000
11590,УКРАИНА,1930,ЯХЬЯЕВА,0.545455


In [261]:
# Number of unique document patterns in fact_data
a = fact_data[["document"]].drop_duplicates().dropna()
a

,document
0,AP!!!!!!
1,AB!!!!!!!
2,СР!!!!!!!
5,!!!!!!!
6,!!!!!!!!!!
...,...
9281,PKGZAC!!!!!!
9341,CH!!!!!!!
9397,!!!!!!!СО
9471,PUZBCB!!!!!!!


In [297]:
# Number of unique document patterns in test_data
a = test_data[["document"]].drop_duplicates().dropna()
a

,document
0,С!!!!!!!
1,!!!!!!!!!!
3,З!!!!!!!
6,С!!!!!!
7,P!!!!!!!
...,...
2405,АН!!!!!!!
2413,ET!!!!!!
2441,CO!!!!!!!
2447,VIIМЮ!!!!!!


In [300]:
# Patterns from test_data not present in fact_data
a = test_data[["document"]].drop_duplicates().dropna()
b = fact_data["document"].dropna().unique()
a[~a["document"].isin(b)]

,document
3,З!!!!!!!
27,СК!!!!!!!!
33,IIСТ!!!!!!
127,АФ!!!!!!!
164,FU!!!!!!
179,Р!!!!!!!!
212,ARM!!!!!!
326,РC!!!!!!!
330,PPP!!!!!!!
358,AZEC!!!!!!


In [304]:
a = fact_data[["document", "country"]].drop_duplicates().dropna()
a.loc[:, "doc_alpha"] = a["document"].str.replace(r"\!+", "")

b = test_data[["document"]].drop_duplicates().dropna()
b.loc[:, "doc_alpha"] = b["document"].str.replace(r"\!+", "")
a

/tmp/ipykernel_213199/694234592.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  a.loc[:, "doc_alpha"] = a["document"].str.replace(r"\!+", "")
/tmp/ipykernel_213199/694234592.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  b.loc[:, "doc_alpha"] = b["document"].str.replace(r"\!+", "")


,document,country,doc_alpha
0,AP!!!!!!,АРМЕНИЯ,AP
1,AB!!!!!!!,УЗБЕКИСТАН,AB
2,СР!!!!!!!,УЗБЕКИСТАН,СР
5,!!!!!!!,КИРГИЗИЯ,
6,!!!!!!!!!!,РОССИЯ,
...,...,...,...
9341,CH!!!!!!!,УЗБЕКИСТАН,CH
9397,!!!!!!!СО,АЗЕРБАЙДЖАН,СО
9448,VИК!!!!!!,АЗЕРБАЙДЖАН,VИК
9471,PUZBCB!!!!!!!,УЗБЕКИСТАН,PUZBCB


In [285]:
unq_doc_lookup[unq_doc_lookup["document"]=="APMBA!!!!!!!" ]

,document,country
343,APMBA!!!!!!!,АРМЕНИЯ


In [258]:
a = test_data[test_data["document"].isin(unq_doc_lookup["document"])].copy()
a = a.drop(columns="country")
pd.merge(a, unq_doc_lookup)

,name,gender,document,name0,name1,name2,name3,name4,name5,country
0,АББАСОВ ЭЛЬФАГ ИБРАГИМ ОГЛЫ,None,P!!!!!!!!,АББАСОВ,ЭЛЬФАГ,ИБРАГИМ,ОГЛЫ,None,None,РОССИЯ
1,АБГАРЯН ДИАНА АРКАДЬЕВНА,None,AR!!!!!!,АБГАРЯН,ДИАНА,АРКАДЬЕВНА,None,None,None,АРМЕНИЯ
2,АЙРОЯН ГЕВОРГ ЭДУАРД,None,AR!!!!!!,АЙРОЯН,ГЕВОРГ,ЭДУАРД,None,None,None,АРМЕНИЯ
3,АМРОЯН КАДЖИК ГЕВОРГОВИЧ,None,AR!!!!!!,АМРОЯН,КАДЖИК,ГЕВОРГОВИЧ,None,None,None,АРМЕНИЯ
4,АНИСЯН АНАИТ КОРЮНОВНА,None,AR!!!!!!,АНИСЯН,АНАИТ,КОРЮНОВНА,None,None,None,АРМЕНИЯ
...,...,...,...,...,...,...,...,...,...,...
563,ЧЕРШУКОВА АНЖЕЛИКА ВЛАДИМИРОВНА,None,АВ!!!!!!!!,ЧЕРШУКОВА,АНЖЕЛИКА,ВЛАДИМИРОВНА,None,None,None,УЗБЕКИСТАН
564,ШИТОВА ОЛЬГА ВАСИЛЬЕВНА,None,ET!!!!!!,ШИТОВА,ОЛЬГА,ВАСИЛЬЕВНА,None,None,None,УКРАИНА
565,ЭВАЙЗОВ АДИЛ ТАНРЫВЕРДИ ОГЛЫ,None,CO!!!!!!!,ЭВАЙЗОВ,АДИЛ,ТАНРЫВЕРДИ,ОГЛЫ,None,None,АЗЕРБАЙДЖАН
566,ЭЙВАЗОВА СААДАТ ЭЛЬЧИН КЫЗЫ,None,VIIМЮ!!!!!!,ЭЙВАЗОВА,СААДАТ,ЭЛЬЧИН,КЫЗЫ,None,None,РОССИЯ


In [267]:
docs = fact_data[["document", "country"]].copy().dropna()
docs.loc[:, "doc_len"] = docs["document"].str.len()
a = docs[["country", "doc_len"]].drop_duplicates()
a.groupby("country").describe()

doc_len                                                
              count    mean       std  min   25%   50%    75%   max
country                                                            
АЗЕРБАЙДЖАН     9.0  10.000  2.738613  6.0  8.00  10.0  12.00  14.0
АРМЕНИЯ         8.0   9.500  2.449490  6.0  7.75   9.5  11.25  13.0
КИРГИЗИЯ        7.0   9.000  2.160247  6.0  7.50   9.0  10.50  12.0
РОССИЯ          7.0   9.000  2.160247  6.0  7.50   9.0  10.50  12.0
УЗБЕКИСТАН      8.0   9.875  2.900123  6.0  7.75   9.5  12.25  14.0
УКРАИНА         6.0   8.500  1.870829  6.0  7.25   8.5   9.75  11.0

Сколько уникальных фамилий из фактического списка "двойного" пола присутствуют в тестовом датасете:

In [287]:
l = 12
a = docs.loc[docs["doc_len"]==l][["country", "document"]].drop_duplicates()
#unq_doc_lookup[unq_doc_lookup["document"].str.len()==l]
a[~(a["document"].isin(unq_doc_lookup["document"]))]

,country,document
2408,РОССИЯ,VIIIМЮ!!!!!!
9115,КИРГИЗИЯ,VIIIМЮ!!!!!!


In [296]:
test_docs = test_data[["document"]].copy().dropna()
test_docs.loc[:, "doc_len"] = test_docs["document"].str.len()
pd.merge(test_docs[test_docs["doc_len"]==12], unq_doc_lookup)

,document,doc_len,country
0,PPAZE!!!!!!!,12,АЗЕРБАЙДЖАН


Присвоим пол по фамилии там где это возможно.

In [127]:
gender_by_name0 = fact_data[["name0", "gender"]].drop_duplicates()
gender_by_name0 = gender_by_name0[~gender_by_name0["name0"].isin(twin_gender["name0"])]
a = test_data.copy()
a = a.drop(columns="gender")
gendered_by_name0 = pd.merge(a, gender_by_name0, on="name0")


In [132]:
a = fact_data[["name1", "gender"]].drop_duplicates()
a = a.dropna()
a = a.groupby("name1").count().reset_index()
twin_gender_name1 = a[a["gender"]>1]
twin_gender_name1

,name1,gender
203,АЙСУН,2
211,АЙХАН,2
390,АРЗУ,2
431,АРШАЛУЙС,2
848,ГУЛЬНАРА,2
882,ГЮНЕЛЬ,2
909,ДАНИИЛ,2
1188,ЗИЯФАТ,2
1454,КЫЗЫ,2
1720,МИХАИЛ,2


In [133]:
fact_data[fact_data["name1"]=="МИХАИЛ"]

,name,gender,document,country,name0,name1,name2,name3,name4,name5
244,АБЕЛОВ МИХАИЛ ВАЛЕРЬЕВИЧ,М,!!!!!!!!!!,РОССИЯ,АБЕЛОВ,МИХАИЛ,ВАЛЕРЬЕВИЧ,None,None,None
281,АБЬЮСЕЙХО МИХАИЛ ЕВГЕНЬЕВИЧ,М,!!!!!!!!!!,РОССИЯ,АБЬЮСЕЙХО,МИХАИЛ,ЕВГЕНЬЕВИЧ,None,None,None
432,АДЫЕВ МИХАИЛ ИВАНОВИЧ,М,!!!!!!!!!!,РОССИЯ,АДЫЕВ,МИХАИЛ,ИВАНОВИЧ,None,None,None
642,АЛЕКСАНДРОВ МИХАИЛ ДМИТРИЕВИЧ,М,!!!!!!!!!!,РОССИЯ,АЛЕКСАНДРОВ,МИХАИЛ,ДМИТРИЕВИЧ,None,None,None
961,АРАКЕЛЯН МИХАИЛ РОБЕРТОВИЧ,М,АН!!!!!!,АРМЕНИЯ,АРАКЕЛЯН,МИХАИЛ,РОБЕРТОВИЧ,None,None,None
1351,БАБКИН МИХАИЛ ТИМОФЕЕВИЧ,М,!!!!!!!!!!,РОССИЯ,БАБКИН,МИХАИЛ,ТИМОФЕЕВИЧ,None,None,None
1555,БАШУК МИХАИЛ АЛЕКСЕЕВИЧ,М,None,УКРАИНА,БАШУК,МИХАИЛ,АЛЕКСЕЕВИЧ,None,None,None
1635,БЕЛЯКОВ МИХАИЛ ЗЕЙНУЛЛА ОГЛЫ,М,!!!!!!!!!!,РОССИЯ,БЕЛЯКОВ,МИХАИЛ,ЗЕЙНУЛЛА,ОГЛЫ,None,None
1683,БИТЬКО МИХАИЛ ВЛАДИМИРОВНА,Ж,ВУ!!!!!!,УКРАИНА,БИТЬКО,МИХАИЛ,ВЛАДИМИРОВНА,None,None,None
1789,БРАЖКО МИХАИЛ ПАВЛОВИЧ,М,СЮ!!!!!!,УКРАИНА,БРАЖКО,МИХАИЛ,ПАВЛОВИЧ,None,None,None


In [121]:
b = test_data.copy()
b.loc[:, "two_names"] = b["name0"] + b["name1"]

len(set(b["two_names"]) - set(a["two_names"]))

2433

In [96]:
fact_data[fact_data["name0"]=="ЩЕРБАК"]

,name,gender,document,country,name0,name1,name2,name3,name4,name5
9234,ЩЕРБАК АНДРЕЙ АЛЕКСЕЕВИЧ,М,!!!!!!!!!!,РОССИЯ,ЩЕРБАК,АНДРЕЙ,АЛЕКСЕЕВИЧ,None,None,None
9235,ЩЕРБАК ДЖУЛЕТА ВРЕЖОВНА,Ж,!!!!!!!!!!,АРМЕНИЯ,ЩЕРБАК,ДЖУЛЕТА,ВРЕЖОВНА,None,None,None
9236,ЩЕРБАК МЕЛИК НИКОЛАЕВИЧ,М,!!!!!!!!!!,УКРАИНА,ЩЕРБАК,МЕЛИК,НИКОЛАЕВИЧ,None,None,None
